<a href="https://colab.research.google.com/github/SinnottKayleigh/B2B-Sales-Algos/blob/main/Micro_Macro_Segmenting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Uses ML to segment customers, examine attributes that may lead to sales
- Industry, region, size
-  Buying patterns - volume, deal size, fx turnover, transaction timing
- To personalise value proposition and prioritise leads
- Pyvis is a python library used to map relationships between variables and data, in a neural network type chart
- 4 ways to segment and understand relationships between client and prospect attributes.

1. Demographic Criteria
2. Operating Variables
3. Purchasing Approaches
4. Situational Factors

- Eliminates the time wasted on low-yield opportunities, provides more accurate information for better decision making; optimising resources and time.

- Cohort Analysis: involves grouping users based on behavioural characteristics, such as churn rate; the identification of patterns among segments.



In [20]:
from pyvis.network import Network
import networkx as nx

Synthetic dataset

In [21]:
import pandas as pd
import numpy as np

np.random.seed(42)

n_customers = 50
customer_ids = [f"CUST_{i+1}" for i in range(n_customers)]
average_trade_sizes = np.random.randint(10000, 1000000, size=n_customers)
credit_ratings = np.random.choice(["AAA", "AA", "A", "BBB"], size=n_customers)
churn_likelihoods = np.round(np.random.uniform(0, 1, size=n_customers), 2)
decision_makers = np.random.choice(["CFO", "Treasury Manager", "CEO", "Head of FX"], size=n_customers)

hq_locations = ["UK"] * n_customers
subsidiary_locations = np.random.choice(["USA", "Germany", "France", "Singapore", "Japan"], size=n_customers)

data = pd.DataFrame({
    "Customer ID": customer_ids,
    "Average Trade Size": average_trade_sizes,
    "HQ Location": hq_locations,
    "Subsidiary Location": subsidiary_locations,
    "Credit Rating": credit_ratings,
    "Churn Likelihood": churn_likelihoods,
    "Key Decision Maker": decision_makers
})

print(data.head())

  Customer ID  Average Trade Size HQ Location Subsidiary Location  \
0      CUST_1              131958          UK                 USA   
1      CUST_2              681155          UK           Singapore   
2      CUST_3              141932          UK               Japan   
3      CUST_4              375838          UK                 USA   
4      CUST_5              269178          UK              France   

  Credit Rating  Churn Likelihood Key Decision Maker  
0            AA              0.84         Head of FX  
1             A              0.45                CEO  
2           BBB              0.40                CFO  
3             A              0.93         Head of FX  
4           BBB              0.73                CFO  


In [22]:
from pyvis.network import Network
import pandas as pd

net = Network(notebook=True, height="750px", width="100%")

for _, row in data.iterrows():
    node_id = row["Customer ID"]
    node_label = f"{node_id}\n({row['Key Decision Maker']})"
    net.add_node(
        node_id,
        label=node_label,
        title=f"""
        Customer ID: {node_id}
        Average Trade Size: ${row['Average Trade Size']:,}
        HQ Location: {row['HQ Location']}
        Subsidiary Location: {row['Subsidiary Location']}
        Credit Rating: {row['Credit Rating']}
        Churn Likelihood: {row['Churn Likelihood']}
        Key Decision Maker: {row['Key Decision Maker']}
        """,
        size=row["Average Trade Size"] / 10000
    )

for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if data.loc[i, "Key Decision Maker"] == data.loc[j, "Key Decision Maker"]:
            net.add_edge(data.loc[i, "Customer ID"], data.loc[j, "Customer ID"], title="Same Decision Maker")

net.toggle_hide_edges_on_drag(True)
net.show_buttons(filter_=['physics'])
net.show("customer_network.html")

customer_network.html


In [23]:
from pyvis.network import Network

net = Network(notebook=True, height="750px", width="100%", cdn_resources='remote')

for _, row in data.iterrows():
    node_id = row["Customer ID"]
    node_label = f"{node_id}\n({row['Key Decision Maker']})"
    net.add_node(
        node_id,
        label=node_label,
        title=f"""
        Customer ID: {node_id}
        Average Trade Size: ${row['Average Trade Size']:,}
        HQ Location: {row['HQ Location']}
        Subsidiary Location: {row['Subsidiary Location']}
        Credit Rating: {row['Credit Rating']}
        Churn Likelihood: {row['Churn Likelihood']}
        Key Decision Maker: {row['Key Decision Maker']}
        """,
        size=row["Average Trade Size"] / 10000
    )

for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if data.loc[i, "Key Decision Maker"] == data.loc[j, "Key Decision Maker"]:
            net.add_edge(data.loc[i, "Customer ID"], data.loc[j, "Customer ID"], title="Same Decision Maker")

net.toggle_hide_edges_on_drag(True)
net.show_buttons(filter_=['physics'])
net.show("customer_network.html")

customer_network.html


In [27]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def create_bubble_chart(data):
    fig = px.scatter(data,
                    x='Average Trade Size',
                    y='Churn Likelihood',
                    size='Average Trade Size',
                    color='Credit Rating',
                    hover_name='Customer ID',
                    text='Key Decision Maker',
                    title='Customer Analysis - Trade Size vs Churn Risk')

    fig.update_traces(textposition='top center')
    return fig

def create_treemap(data):
    fig = px.treemap(data,
                    path=['HQ Location', 'Subsidiary Location', 'Key Decision Maker', 'Customer ID'],
                    values='Average Trade Size',
                    color='Credit Rating',
                    title='Customer Hierarchy by Location and Decision Maker')
    return fig

def create_dashboard(data):
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Trade Size Distribution', 'Decision Maker Distribution',
                       'Credit Rating Distribution', 'Churn Risk Distribution')
    )

    fig.add_trace(
        go.Histogram(x=data['Average Trade Size'], name='Trade Size'),
        row=1, col=1
    )

    decision_maker_counts = data['Key Decision Maker'].value_counts()
    fig.add_trace(
        go.Bar(x=decision_maker_counts.index, y=decision_maker_counts.values,
               name='Decision Makers'),
        row=1, col=2
    )

    credit_rating_counts = data['Credit Rating'].value_counts()
    fig.add_trace(
        go.Bar(x=credit_rating_counts.index, y=credit_rating_counts.values,
               name='Credit Ratings'),
        row=2, col=1
    )

    fig.add_trace(
        go.Histogram(x=data['Churn Likelihood'], name='Churn Risk'),
        row=2, col=2
    )

    fig.update_layout(height=800,
                     showlegend=True,
                     title_text="Customer Analysis Dashboard")

    fig.update_xaxes(title_text="Trade Size", row=1, col=1)
    fig.update_xaxes(title_text="Decision Maker", row=1, col=2)
    fig.update_xaxes(title_text="Credit Rating", row=2, col=1)
    fig.update_xaxes(title_text="Churn Likelihood", row=2, col=2)

    fig.update_yaxes(title_text="Count", row=1, col=1)
    fig.update_yaxes(title_text="Count", row=1, col=2)
    fig.update_yaxes(title_text="Count", row=2, col=1)
    fig.update_yaxes(title_text="Count", row=2, col=2)

    return fig

def create_decision_maker_pie(data):
    decision_maker_counts = data['Key Decision Maker'].value_counts()
    fig = go.Figure(data=[go.Pie(labels=decision_maker_counts.index,
                                values=decision_maker_counts.values,
                                title='Decision Maker Distribution')])
    return fig

bubble_chart = create_bubble_chart(data)
treemap = create_treemap(data)
dashboard = create_dashboard(data)
decision_maker_pie = create_decision_maker_pie(data)
bubble_chart.show()
treemap.show()
dashboard.show()
decision_maker_pie.show()